# ESMFold breakdown
It took ouput from ESM (sequence reperesentation, attention matrix) as input
into folding trunk then structure module to predict structure.  

utility modules (misc.py)
read seqence module (encode_sequence): input sequence, add chain_linker( optional, default 'G'*25), add residue_index_offset (add a jump in index if in different chain)
return:encoded (residue token), residx, linker_mask, chain_index

read seqence in batch 

output_to_pdb: input: output from model (in dict format) output: pdb files

Attention: to calcualte attention in Transformer

Dropout: 

SequencToPair:

PairToSequence:





## This is how differnt ESM model get loaded into ESMFold model

In [1]:
def _load_model(model_name):
    if model_name.endswith(".pt"):  # local, treat as filepath
        model_path = Path(model_name)
        model_data = torch.load(str(model_path), map_location="cpu")
    else:  # load from hub
        url = f"https://dl.fbaipublicfiles.com/fair-esm/models/{model_name}.pt"
        model_data = torch.hub.load_state_dict_from_url(url, progress=False, map_location="cpu")

    cfg = model_data["cfg"]["model"]
    model_state = model_data["model"]  # contain weights and bais
    model = ESMFold(esmfold_config=cfg)

    expected_keys = set(model.state_dict().keys())
    found_keys = set(model_state.keys())

    missing_essential_keys = []  # make sure keys are compatiable before load parameters
    for missing_key in expected_keys - found_keys:
        if not missing_key.startswith("esm."):
            missing_essential_keys.append(missing_key)

    if missing_essential_keys:
        raise RuntimeError(f"Keys '{', '.join(missing_essential_keys)}' are missing.")

    model.load_state_dict(model_state, strict=False) # 

    return model